# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")

### Get data from `.csv` files and then preprocess data

In [2]:
def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    # Read English movie reviews
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_eng = df_eng.rename(columns={'Movie / TV Series': 'Title', 'Year': 'Year', 'Synopsis': 'Synopsis', 'Review': 'Review'})
    df_eng['Original Language'] = 'English'
    
    # Read French movie reviews
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr = df_fr.rename(columns={'Titre': 'Title', 'Année': 'Year', 'Synopsis': 'Synopsis', 'Critiques': 'Review'})
    df_fr['Original Language'] = 'French'
    
    # Read Spanish movie reviews
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp = df_sp.rename(columns={'Título': 'Title', 'Año': 'Year', 'Sinopsis': 'Synopsis', 'Críticas': 'Review'})
    df_sp['Original Language'] = 'Spanish'
    
    # Concatenate all dataframes into one resultant dataframe called "df"
    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    
    return df

df = preprocess_data()

In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",French
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
25,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",Spanish
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
26,Toc Toc,2017,"En esta comedia española, un grupo de personas...","""Toc Toc es una película aburrida y poco origi...",Spanish
22,Y tu mamá también,2001,Dos amigos adolescentes (Gael García Bernal y ...,"""Y tu mamá también es una película que se qued...",Spanish


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# Define the model names for the translation tasks
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"

# Create the models and tokenizers
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

In [5]:

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0],skip_special_tokens=True)
    return decoded

In [6]:
# Filter reviews in French and translate them to English
df_fr = df[df["Original Language"] == "French"].copy()
fr_reviews = df_fr["Review"]
fr_reviews_en = fr_reviews.apply(lambda fr_reviews: translate(fr_reviews, fr_en_model,fr_en_tokenizer))

fr_synopsis = df_fr["Synopsis"]
fr_synopsis_en = fr_synopsis.apply(lambda fr_synopsis: translate(fr_synopsis, fr_en_model,fr_en_tokenizer))


# filter reviews in Spanish and translate to English
df_es = df[df["Original Language"] == "Spanish"]
es_reviews = df_es["Review"]
es_reviews_en = es_reviews.apply(lambda es_reviews: translate(es_reviews, es_en_model,es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df_es["Synopsis"]
es_synopsis_en = es_synopsis.apply(lambda es_synopsis: translate(es_synopsis, es_en_model,es_en_tokenizer))


In [7]:
# Filter out French reviews and synopses
df_filtered = df[df["Original Language"] != "French"]

# Update DataFrame with translated reviews and synopses for French
df_fr["Review"] = fr_reviews_en
df_fr["Synopsis"] = fr_synopsis_en

# Filter out Spanish reviews and synopses
df_filtered = df_filtered[df_filtered["Original Language"] != "Spanish"]

# Update DataFrame with translated reviews and synopses for Spanish
df_es["Review"] = es_reviews_en
df_es["Synopsis"] = es_synopsis_en

# Concatenate the modified DataFrames (English reviews and synopses) with the filtered DataFrame
df = pd.concat([df_filtered, df_fr, df_es], ignore_index=True)

In [9]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",Spanish
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",French
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",Spanish
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French
15,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ...",French
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [10]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model= model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    result = classifier(text)
    return result

In [13]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
reviews = df["Review"]
results = reviews.apply(lambda reviews: analyze_sentiment(reviews,sentiment_classifier))
df["Sentiment"] = results.apply(lambda results: results[0]['label'])

In [14]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,sentiment,Sentiment
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish,NEGATIVE,NEGATIVE
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French,POSITIVE,POSITIVE
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English,NEGATIVE,NEGATIVE
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",French,POSITIVE,POSITIVE
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",French,NEGATIVE,NEGATIVE
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",Spanish,POSITIVE,POSITIVE
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",Spanish,NEGATIVE,NEGATIVE
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",Spanish,NEGATIVE,NEGATIVE
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",Spanish,POSITIVE,POSITIVE
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Spanish,POSITIVE,POSITIVE


In [16]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)